# Networking (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

Installing Packages Expecto

Installing Packages Expecto.

Installing Packages Expecto..

Installing Packages Expecto...

Installing Packages Expecto....

Installing Packages Expecto.....

Installing Packages Expecto......

Installing Packages Expecto.......

Installed Packages Expecto, 10.1.0

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/Async.fs

In [ ]:
open Common

## testPortOpen

In [ ]:
let testPortOpen port = async {
    let! ct = Async.CancellationToken
    use client = new System.Net.Sockets.TcpClient ()
    try
        do! client.ConnectAsync ("127.0.0.1", port, ct) |> Async.awaitValueTaskUnit
        return true
    with ex ->
        trace Verbose (fun () -> $"testPortOpen / ex: {ex |> printException}") getLocals
        return false
}

In [ ]:
//// test

testPortOpen 65536
|> Async.runWithTimeout 60
|> _equal (Some false)

17:25:22 #1 [Verbose] testPortOpen / ex: System.ArgumentOutOfRangeException: Specified argument was out of the range of valid values. (Parameter 'port')
FSharpOption<Boolean>
      Value: False


In [ ]:
let testPortOpenTimeout timeout port = async {
    let! result =
        testPortOpen port
        |> Async.runWithTimeoutAsync timeout
    return
        match result with
        | None -> false
        | Some result -> result
}

In [ ]:
//// test

testPortOpenTimeout 60 65535
|> Async.RunSynchronously
|> _equal false

17:25:23 #2 [Debug] runWithTimeoutAsync / timeout: 60
False


## waitForPortAccess

In [ ]:
let waitForPortAccess timeout status port =
    let rec loop retry = async {
        let! isPortOpen =
            match timeout with
            | None -> testPortOpen port
            | Some timeout -> testPortOpenTimeout timeout port
        if isPortOpen = status
        then return retry
        else
            if retry % 100 = 0 then
                let getLocals () = $"port: {port} / retry: {retry} / {getLocals ()}"
                trace Warn (fun () -> "waitForPortAccess") getLocals
            do! Async.Sleep 1
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let port = 5000

let lockPort () = async {
    trace Debug (fun () -> "_1") getLocals
    do! Async.Sleep 5000
    let listener = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    trace Debug (fun () -> "_2") getLocals
    listener.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_4") getLocals
    listener.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! _ = lockPort () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "3") getLocals
    let! retries1 = waitForPortAccess None true port
    let! retries2 = waitForPortAccess None false port
    return retries1, retries2
}
|> Async.runWithTimeout 20000
|> function
    | Some (retries1, retries2) ->
        retries1 |> _isGreaterThanOrEqual 2
        retries2 |> _isGreaterThanOrEqual 100
        true
    | _ -> false
|> _equal true

17:25:23 #3 [Debug] 1
17:25:23 #4 [Debug] _1
17:25:23 #5 [Debug] 2
17:25:23 #6 [Debug] 3
17:25:25 #7 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
17:25:25 #8 [Warn] waitForPortAccess / port: 5000 / retry: 0
17:25:27 #9 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
17:25:28 #10 [Debug] _2
17:25:28 #11 [Debug] _3
17:25:28 #12 [Warn] waitForPortAccess / port: 5000 / retry: 0
17:25:29 #13 [Warn] waitForPortAccess / port: 5000 / retry: 100
17:25:30 #14 [Debug] _4
17:25:30 #15 [Debug] _5
17:25:32 #16 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
2
120
True


In [ ]:
//// test

let port = 5000

let lockPort () = async {
    trace Debug (fun () -> "_1") getLocals
    do! Async.Sleep 500
    let listener = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    trace Debug (fun () -> "_2") getLocals
    listener.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 200
    trace Debug (fun () -> "_4") getLocals
    listener.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! _ = lockPort () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "3") getLocals
    let! retries1 = waitForPortAccess (Some 60) true port
    let! retries2 = waitForPortAccess (Some 60) false port
    return retries1, retries2
}
|> Async.runWithTimeout 2000
|> function
    | Some (retries1, retries2) ->
        retries1 |> _isGreaterThanOrEqual 2
        retries2 |> _isGreaterThanOrEqual 10
        true
    | _ -> false
|> _equal true

17:25:32 #17 [Debug] 1
17:25:32 #18 [Debug] 2
17:25:32 #19 [Debug] _1
17:25:32 #20 [Debug] 3
17:25:32 #21 [Debug] runWithTimeoutAsync / timeout: 60
17:25:32 #22 [Warn] waitForPortAccess / port: 5000 / retry: 0
17:25:32 #23 [Debug] runWithTimeoutAsync / timeout: 60
17:25:32 #24 [Debug] runWithTimeoutAsync / timeout: 60
17:25:32 #25 [Debug] runWithTimeoutAsync / timeout: 60
17:25:32 #26 [Debug] runWithTimeoutAsync / timeout: 60
17:25:32 #27 [Debug] runWithTimeoutAsync / timeout: 60
17:25:33 #28 [Debug] _2
17:25:33 #29 [Debug] _3
17:25:33 #30 [Debug] runWithTimeoutAsync / timeout: 60
17:25:33 #31 [Warn] waitForPortAccess / port: 5000 / retry: 0
17:25:33 #32 [Debug] _4
17:25:33 #33 [Debug] _5
17:25:33 #34 [Debug] runWithTimeoutAsync / timeout: 60
7
10
True


## getAvailablePort

In [ ]:
let getAvailablePort timeout initialPort =
    let rec loop port = async {
        let! isPortOpen =
            match timeout with
            | None -> testPortOpen port
            | Some timeout -> testPortOpenTimeout timeout port
        if not isPortOpen
        then return port
        else return! loop (port + 1)
    }
    loop initialPort

In [ ]:
//// test

let port = 5000

let lockPorts () = async {
    trace Debug (fun () -> "_1") getLocals
    let listener1 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    let listener2 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port + 1)
    trace Debug (fun () -> "_2") getLocals
    listener1.Start ()
    listener2.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 4000
    trace Debug (fun () -> "_4") getLocals
    listener1.Stop ()
    listener2.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! _ = lockPorts () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "3") getLocals
    let! availablePort = getAvailablePort None port
    let! retries = waitForPortAccess None false port
    return availablePort, retries
}
|> Async.runWithTimeout 15000
|> function
    | Some (availablePort, retries) ->
        availablePort |> _equal (port + 2)
        retries |> _isGreaterThanOrEqual 100
        true
    | _ -> false
|> _equal true

17:25:33 #35 [Debug] 1
17:25:33 #36 [Debug] 2
17:25:33 #37 [Debug] _1
17:25:33 #38 [Debug] _2
17:25:33 #39 [Debug] _3
17:25:33 #40 [Debug] 3
17:25:35 #41 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
17:25:35 #42 [Warn] waitForPortAccess / port: 5000 / retry: 0
17:25:37 #43 [Warn] waitForPortAccess / port: 5000 / retry: 100
17:25:37 #44 [Debug] _4
17:25:37 #45 [Debug] _5
17:25:39 #46 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
5002
124
True


In [ ]:
//// test

let port = 5000

let lockPorts () = async {
    trace Debug (fun () -> "_1") getLocals
    let listener1 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    let listener2 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port + 1)
    trace Debug (fun () -> "_2") getLocals
    listener1.Start ()
    listener2.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 400
    trace Debug (fun () -> "_4") getLocals
    listener1.Stop ()
    listener2.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! _ = lockPorts () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "3") getLocals
    let! availablePort = getAvailablePort (Some 60) port
    let! retries = waitForPortAccess (Some 60) false port
    return availablePort, retries
}
|> Async.runWithTimeout 1500
|> function
    | Some (availablePort, retries) ->
        availablePort |> _equal (port + 2)
        retries |> _isGreaterThanOrEqual 10
        true
    | _ -> false
|> _equal true

17:25:39 #47 [Debug] 1
17:25:39 #48 [Debug] 2
17:25:39 #49 [Debug] _1
17:25:39 #50 [Debug] _2
17:25:39 #51 [Debug] _3
17:25:39 #52 [Debug] 3
17:25:39 #53 [Debug] runWithTimeoutAsync / timeout: 60
17:25:39 #54 [Warn] waitForPortAccess / port: 5000 / retry: 0
17:25:40 #55 [Debug] _4
17:25:40 #56 [Debug] _5
17:25:40 #57 [Debug] runWithTimeoutAsync / timeout: 60
5002
22
True
